In [81]:
import pandas as pd
import torch
import os
# Set working directory

try:
    data_train = pd.read_csv('dataset/ogbn_mag/split/time/paper/train.csv.gz', compression='gzip',header = None)
    data_valid = pd.read_csv('dataset/ogbn_mag/split/time/paper/valid.csv.gz', compression='gzip',header = None)
    data_test = pd.read_csv('dataset/ogbn_mag/split/time/paper/test.csv.gz', compression='gzip',header = None)
except FileNotFoundError:
    os.chdir("..")
    data_train = pd.read_csv('dataset/ogbn_mag/split/time/paper/train.csv.gz', compression='gzip',header = None)
    data_valid = pd.read_csv('dataset/ogbn_mag/split/time/paper/valid.csv.gz', compression='gzip',header = None)
    data_test = pd.read_csv('dataset/ogbn_mag/split/time/paper/test.csv.gz', compression='gzip',header = None)

data, _ = torch.load(r"dataset/ogbn_mag/processed/geometric_data_processed.pt", weights_only=False)

# Extract edges for "paper" -> "cites" -> "paper"
paper_c_paper = data.edge_index_dict[('paper', 'cites', 'paper')]

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_c_paper[0], nums_train) | torch.isin(paper_c_paper[1], nums_train)
mask_valid = torch.isin(paper_c_paper[0], nums_valid) | torch.isin(paper_c_paper[1], nums_valid)
mask_test = torch.isin(paper_c_paper[0], nums_test) | torch.isin(paper_c_paper[1], nums_test)

paper_c_paper_train = paper_c_paper.clone()
paper_c_paper_valid = paper_c_paper.clone()
paper_c_paper_test = paper_c_paper.clone()

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_c_paper_train = paper_c_paper_train[:, mask_train_done]
paper_c_paper_valid = paper_c_paper_valid[:, mask_valid_done]
paper_c_paper_test = paper_c_paper_test[:, mask_test]

len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1]


(5416271, 5416271)

In [60]:
author_w_paper = data.edge_index_dict[('author', 'writes', 'paper')]

author_w_paper_train = author_w_paper.clone()
author_w_paper_valid = author_w_paper.clone()
author_w_paper_test = author_w_paper.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(author_w_paper[1], nums_train)
mask_valid = torch.isin(author_w_paper[1], nums_valid)
mask_test = torch.isin(author_w_paper[1], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
author_w_paper_train = author_w_paper_train[:, mask_train_done]
author_w_paper_valid = author_w_paper_valid[:, mask_valid_done]
author_w_paper_test = author_w_paper_test[:, mask_test]

len(author_w_paper_train[1]) + len(author_w_paper_valid[1]) + len(author_w_paper_test[1]), author_w_paper.shape[1]


(7145660, 7145660)

In [3]:
paper_t_field = data.edge_index_dict[('paper', 'has_topic', 'field_of_study')]

paper_t_field_train = paper_t_field.clone()
paper_t_field_valid = paper_t_field.clone()
paper_t_field_test = paper_t_field.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_t_field[0], nums_train)
mask_valid = torch.isin(paper_t_field[0], nums_valid)
mask_test = torch.isin(paper_t_field[0], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_t_field_train = paper_t_field_train[:, mask_train_done]
paper_t_field_valid = paper_t_field_valid[:, mask_valid_done]
paper_t_field_test = paper_t_field_test[:, mask_test]

len(paper_t_field_train[1]) + len(paper_t_field_valid[1]) + len(paper_t_field_test[1]), paper_t_field.shape[1]


(7505078, 7505078)

In [ ]:
data["y_dict"]["paper"] = pd.DataFrame(data["y_dict"]["paper"]).reset_index()
data["y_dict"]["paper"].columns = ['index', 'value']

#convert to tensor
paper_y = torch.tensor(data["y_dict"]["paper"]['value'].values)
paper_y_train = paper_y.clone()
paper_y_valid = paper_y.clone()
paper_y_test = paper_y.clone()

KeyError: tensor([True, True, True,  ..., True, True, True])

In [86]:
data["y_dict"]["paper"][2],data["y_dict"]["paper"][5],data["y_dict"]["paper"][4]

(tensor([189]), tensor([9]), tensor([95]))

In [56]:
import numpy as np

paper_c_paper_train_0 = np.array(paper_c_paper_train[0])
paper_c_paper_train_1 = np.array(paper_c_paper_train[1])

datamatrix = np.column_stack([
    (paper_c_paper_train_0 == np.arange(len(paper_c_paper_train_0))).astype(int), paper_c_paper_train_0, paper_c_paper_train_1, 
])

# np.argwhere(d[0, 1] == 0)
len(np.argwhere(datamatrix[:, 0] == 0)), len(np.argwhere(datamatrix[:, 0] == 1))

/tmp/ipykernel_94203/2906774071.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  paper_c_paper_train_0 = np.array(paper_c_paper_train[0])
/tmp/ipykernel_94203/2906774071.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  paper_c_paper_train_1 = np.array(paper_c_paper_train[1])


(3879967, 1)

tensor([[     0,      0,      0,  ...,    184,    184,    185],
        [    88,  27449, 121051,  ..., 497699, 720312,  96064]])

In [11]:
import numpy as np
from scipy.sparse import coo_matrix
import torch

# only use the first 1000 papers
max_paper_id = 1000
paper_c_paper_train_sampled = paper_c_paper_train[:, :max_paper_id]

# Example citation data: Replace with your actual data
# For example:
# paper_ids = [0, 88, 27449, 121051, ...]  # Citing papers
# cited_ids = [88, 27449, 121051, ..., 421711, 427339, 439864]  # Cited papers

# Example tensor as you mentioned (2 rows, multiple columns)
# tensor[0] represents paper_ids (papers that cite others)
# tensor[1] represents cited paper_ids (papers being cited)
tensor = paper_c_paper_train_sampled

# Extract the data
paper_ids = tensor[0].numpy()  # Citing papers
cited_ids = tensor[1].numpy()  # Cited papers

# Create a sparse matrix in COO format:
citation_values = np.ones(len(paper_ids))  # All citations will have value 1

# Create the sparse matrix with the correct shape
citation_matrix_coo = coo_matrix((citation_values, (paper_ids, cited_ids)), shape=(max(paper_ids) + 1, max(cited_ids) + 1))

# Now you have a sparse matrix with 3 columns (paper_id, cited_paper_id, 1 for citation)
# Optionally, you can convert it to CSR format for better performance:
citation_matrix_csr = citation_matrix_coo.tocsr()

# If you want to save this matrix to disk (e.g., in HDF5 format), you can do so:
# import h5py
# with h5py.File('citation_matrix.h5', 'w') as f:
#     f.create_dataset('rows', data=citation_matrix_coo.row)
#     f.create_dataset('cols', data=citation_matrix_coo.col)
#     f.create_dataset('data', data=citation_matrix_coo.data)


# use in pytorch
citation_matrix = torch.sparse_coo_tensor((citation_matrix_coo.row, citation_matrix_coo.col), citation_matrix_coo.data)


In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
import torch

# Only use the first 1000 papers
max_paper_id = 1000
paper_c_paper_train_sampled = paper_c_paper_train[:, :max_paper_id]

# Extract the data
paper_ids = paper_c_paper_train_sampled[0].numpy()  # Citing papers
cited_ids = paper_c_paper_train_sampled[1].numpy()  # Cited papers

# Filter to ensure indices are within range
valid_mask = (paper_ids <= max_paper_id) & (cited_ids <= max_paper_id)
paper_ids = paper_ids[valid_mask]
cited_ids = cited_ids[valid_mask]

# Create a sparse matrix in COO format with value 1 for citations
citation_values = np.ones(len(paper_ids))  # All citations have value 1
citation_matrix_coo = coo_matrix(
    (citation_values, (paper_ids, cited_ids)),
    shape=(max_paper_id + 1, max_paper_id + 1)
)

# Convert to dense format to include missing links with label 0
full_citation_matrix = citation_matrix_coo.toarray()

# Convert back to sparse format for efficiency
citation_matrix_coo = coo_matrix(full_citation_matrix)

# Use in PyTorch
citation_matrix = torch.sparse_coo_tensor(
    (citation_matrix_coo.row, citation_matrix_coo.col), citation_matrix_coo.data
)


NotImplementedError: Could not run 'aten::as_strided' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::as_strided' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, Fun

In [19]:
import torch

class LossFunction:
    def __init__(self, alpha=1.0, eps=1e-8, use_regularization=False, lam=0.001):
        """
        Initialize the loss function with given parameters.
        
        Args:
            alpha (float): Scaling parameter for edge probability.
            eps (float): Small value to prevent log(0).
            use_regularization (bool): Whether to include Gaussian regularization.
        """
        self.alpha = alpha
        self.eps = eps
        self.lam = lam
        self.use_regularization = use_regularization

    def edge_probability(self, z_i, z_j):
        """Compute the probability of an edge existing between two embeddings."""
        dist = torch.norm(z_i - z_j) ** 2  # Squared Euclidean distance
        exponent = torch.clamp(-self.alpha + dist, min=-50, max=50)  # Prevents extreme values
        return 1 / (1 + torch.exp(exponent))  # Logistic function

    def link_loss(self, label, z_u, z_v):
        """Compute the loss for a single edge."""
        prob = self.edge_probability(z_u, z_v)
        prob = torch.clamp(prob, self.eps, 1 - self.eps)  # Numerical stability
        return label.float() * torch.log(prob) + (1 - label.float()) * torch.log(1 - prob)

    def compute_loss(self, node_embeddings, datamatrix_tensor):
        """Compute the total loss for the dataset using sparse tensor."""
        
        # Extract indices and values from the sparse tensor
        indices = datamatrix_tensor.coalesce().indices()  # Shape: [2, nnz]
        values = datamatrix_tensor.coalesce().values()    # Shape: [nnz]

        u_idx = indices[0]  # Citing papers
        v_idx = indices[1]  # Cited papers
        
        # Assume the indices are in the form of (u_idx, v_idx) pairs
        z_u = node_embeddings(u_idx)  # Lookup embeddings
        z_v = node_embeddings(v_idx)
        
        # Compute the link loss for the pairs (u_idx, v_idx) using vectorized operations
        # self.link_loss should support batch operations for efficiency
        losses = self.link_loss(values, z_u, z_v)
        # Sum the losses and compute the final loss
        sum_loss = losses.sum()  # Sum over all citation events
        loss = -sum_loss / len(values)  # Divide by the number of non-zero values (citations)
        
        return loss
    

# loss_fn = LossFunction(alpha=1.0, use_regularization=True)
# loss_value = loss_fn.compute_loss(z, datamatrix_tensor)

In [21]:
citation_matrix.coalesce().indices().shape

torch.Size([2, 18])

In [14]:
embedding_dim = 2
node_embeddings = torch.nn.Embedding((citation_matrix.coalesce().indices().shape[1]), embedding_dim)


loss_function = LossFunction(alpha=5, eps=1e-2, use_regularization=False)

optimizer = torch.optim.Adam(node_embeddings.parameters(), lr=0.1)

alpha = 5
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = loss_function.compute_loss(node_embeddings, citation_matrix)  # Compute loss
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

IndexError: index out of range in self

In [12]:
import numpy as np
import torch
from scipy.sparse import coo_matrix

# Assume paper_c_paper_train is a tensor with shape (2, num_links)
tensor = paper_c_paper_train

# Extract existing citations
paper_ids = tensor[0].numpy()  # Citing papers
cited_ids = tensor[1].numpy()  # Cited papers

num_papers = max(max(paper_ids), max(cited_ids)) + 1  # Total number of papers

# Step 1: Create Positive (Existing) Citations
citation_values = np.ones(len(paper_ids))  # All existing citations have value 1
citation_matrix_coo = coo_matrix((citation_values, (paper_ids, cited_ids)), shape=(num_papers, num_papers))

# Step 2: Generate Negative Samples (Missing Links)
num_neg_samples = 100  # Aim for a 1:1 ratio of positive to negative
negative_samples = set()

while len(negative_samples) < num_neg_samples:
    i = np.random.randint(0, num_papers)
    j = np.random.randint(0, num_papers)
    if i != j and (i, j) not in zip(paper_ids, cited_ids):  # Ensure no duplicates
        negative_samples.add((i, j))

neg_paper_ids, neg_cited_ids = zip(*negative_samples)
neg_values = np.zeros(len(neg_paper_ids))  # Missing links have value 0

# Step 3: Merge Positive and Negative Samples
all_paper_ids = np.concatenate([paper_ids, np.array(neg_paper_ids)])
all_cited_ids = np.concatenate([cited_ids, np.array(neg_cited_ids)])
all_values = np.concatenate([citation_values, neg_values])

# Create a sparse matrix including both 1s and 0s
citation_matrix_coo = coo_matrix((all_values, (all_paper_ids, all_cited_ids)), shape=(num_papers, num_papers))

# Step 4: Convert to PyTorch Sparse Tensor
indices = torch.tensor([citation_matrix_coo.row, citation_matrix_coo.col], dtype=torch.long)
values = torch.tensor(citation_matrix_coo.data, dtype=torch.float32)

citation_matrix = torch.sparse_coo_tensor(indices, values, torch.Size([num_papers, num_papers]))



In [ ]:
paper_c_paper_train[:,88]

tensor([    28, 333019])

In [ ]:
print(np.argwhere(paper_c_paper_train[0] == 2))
print(paper_c_paper_train[1][5])

tensor([[5, 6, 7]])
tensor(186851)


In [ ]:
# creating datamatrix where 1 if link 0 if not, also add the 1st and 2nd column to the datamatrix
from tqdm import tqdm
datamatrix = []
for i in tqdm(range(len(paper_c_paper_train[0]))):
    label = 1 if i == paper_c_paper_train[0][i] else 0
    datamatrix.append([paper_c_paper_train[0][i], paper_c_paper_train[1][i], label])

print(datamatrix)

  0%|          | 0/3879968 [00:00<?, ?it/s]

100%|██████████| 3879968/3879968 [00:57<00:00, 67341.46it/s]


KeyboardInterrupt: 